In [ ]:
from moviepy.editor import VideoFileClip
from IPython.display import HTML
import numpy as np
import cv2
import pickle
import glob

from utilities import *

In [ ]:
dist_pickle = pickle.load(open('camera_cal/calibration.p','rb'))
mtx = dist_pickle['mtx']
dist = dist_pickle['dist']

In [ ]:

Output_video = 'output1_tracked.mp4'
Input_video = 'project_video.mp4'

clip1 = VideoFileClip(Input_video)
video_clip = clip1.fl_image(lambda image: process_image(img=image, mtx=mtx,dist=dist))
video_clip.write_videofile(Output_video, audio=False)